In [1]:
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv('../../input/clean_data/bici_clean.csv')

In [3]:
data.drop(['fecha', 'dia', 'year', 'tipo_accidente', 'distrito', 'direccion'],axis=1, inplace = True)

In [4]:
data[data.festividad == 'Vispera']

,horario,festividad,lesividad,meteo,lon,lat
7,Tarde,Vispera,Leve,Despejado,-3.630845,40.406749
41,Noche,Vispera,Leve,Despejado,-3.678759,40.437267
54,Noche,Vispera,Moderada,Despejado,-3.686912,40.437669
66,Noche,Vispera,Leve,Despejado,-3.705046,40.411383
85,Tarde,Vispera,Leve,Despejado,-3.702248,40.405124
...,...,...,...,...,...,...
2147,Tarde,Vispera,Leve,Despejado,-3.719200,40.432376
2162,Tarde,Vispera,Leve,Despejado,-3.690617,40.408637
2165,Noche,Vispera,Leve,Despejado,-3.705760,40.429385
2176,Tarde,Vispera,Grave,Despejado,-3.600571,40.436147


In [5]:
def num_horario(value):
    dict_hora = {
        'Mañana':1,
        'Tarde':2,
        'Noche':3
    }
    return dict_hora[value]

def num_fest(value):
    dict_fest = {
        'Laborable':1,
        'Vispera':2,
        'Festivo':3
    }
    return dict_fest[value]

In [6]:
data.horario = data.horario.apply(lambda x: num_horario(x))
data.festividad = data.festividad.apply(lambda x: num_fest(x))
data = pd.get_dummies(data, columns=['meteo'])

In [7]:
data.head()

,horario,festividad,lesividad,lon,lat,meteo_Despejado,meteo_Lluvia,meteo_Niebla
0,2,3,Moderada,-3.681209,40.428990,1,0,0
1,3,1,Grave,-3.654527,40.466921,1,0,0
2,2,1,Leve,-3.623423,40.381971,1,0,0
3,2,1,Leve,-3.608276,40.377939,1,0,0
4,2,1,Leve,-3.608276,40.377939,1,0,0


In [20]:
X = data.drop(['lesividad'], axis=1)
y = data.lesividad
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
# rfc = RandomForestClassifier(n_estimators=100, n_jobs=-1)
# rfc.fit(X_train, y_train)
# y_pred = rfc.predict(X_test)

# params = rfc.get_params

# dpre=pd.DataFrame({
#     'pred':y_pred,
#     'GT':y_test})

# acierto = round((dpre[dpre.pred == dpre.GT].shape[0]/len(dpre.pred))*100,2)

# with open('../../output/pred/records.txt', "a+") as file:
#     file.write(
#         f'''Model: RFC\t Acierto: {acierto}%\t Params: {params} \n\n'''
#     )

In [22]:
knc = KNeighborsClassifier(n_jobs=-1)
knc.fit(X_train, y_train)
y_pred = knc.predict(X_test)

params = knc.get_params

dpre=pd.DataFrame({
    'pred':y_pred,
    'GT':y_test})

acierto = round((dpre[dpre.pred == dpre.GT].shape[0]/len(dpre.pred))*100,2)

with open('../../output/pred/records.txt', "a+") as file:
    file.write(
        f'''Model: KNC\t Acierto: {acierto}%\t Params: {params} \n\n'''
    )

In [ ]:
# Save the trained model as a pickle string. 
saved_model = pickle.dumps(rfc) 
  
# Load the pickled model 
# knn_from_pickle = pickle.loads(saved_model) 
  
# Use the loaded pickled model to make predictions 
# knn_from_pickle.predict(X_test) 